<small><font color=gray>Notebook author: <a href="https://www.linkedin.com/in/olegmelnikov/" target="_blank">Oleg Melnikov</a> ©2021 onwards</font></small><hr style="margin:0;background-color:silver">

**[<font size=6>🚗Auto</font>](https://www.kaggle.com/competitions/13nov23hse-auto/rules)**. [**Instructions**](https://colab.research.google.com/drive/1riOGrE_Fv-yfIbM5V4pgJx4DWcd92cZr#scrollTo=ITaPDPIQEgXV) for running Colabs.

<small>**(Optional) CONSENT.** <mark>[ X ]</mark> We consent to sharing our Colab (after the assignment ends) with other students/instructors for educational purposes. We understand that sharing is optional and this decision will not affect our grade in any way. <font color=gray><i>(If ok with sharing your Colab for educational purposes, leave "X" in the check box.)</i></font></small>

In [ ]:
%%time
%reset -f
from IPython.core.interactiveshell import InteractiveShell as IS; IS.ast_node_interactivity = "all"
import pandas as pd, time, numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge
ToCSV = lambda df, fname: df.round(2).to_csv(f'{fname}.csv', index_label='id') # rounds values to 2 decimals

class Timer():
  def __init__(self, lim:'RunTimeLimit'=60): self.t0, self.lim, _ = time.time(), lim, print('timer started')
  def ShowTime(self):
    msg = f'Runtime is {time.time()-self.t0:.0f} sec'
    print(f'\033[91m\033[1m' + msg + f' > {self.lim} sec limit!!!\033[0m' if (time.time()-self.t0-1) > self.lim else msg)

np.set_printoptions(linewidth=10000, precision=4, edgeitems=20, suppress=True)
pd.set_option('display.max_rows', 100, 'display.max_columns', 100, 'display.max_colwidth', 100, 'display.precision', 2, 'display.max_rows', 4)

db = fetch_openml('BNG(auto_price)')   # load databunch (dictionary)
tX = pd.DataFrame(db['data'], columns=db['feature_names'])
tX.symboling = tX.symboling.astype('float')
tX['price'] = db['target']
YCols = ['city-mpg','highway-mpg','price']  # 3 targets
tY = tX[YCols]
tX.drop(YCols, axis=1, inplace=True)
# tY = pd.Series(db['target'], name='price')
tX, vX, tY, DO_NOT_USE = train_test_split(tX, tY, train_size=0.7, random_state=0, shuffle=True)
# ToCSV(DO_NOT_USE, 'testY')   # Students cannot use these test values
del DO_NOT_USE
tX
tY
tmr = Timer() # runtime limit (in seconds). Add all of your code after the timer

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


timer started
CPU times: user 23 s, sys: 1.54 s, total: 24.6 s
Wall time: 44.5 s


In [ ]:
tmr = Timer()

timer started


<hr color=red>

<font size=5>⏳</font> <strong><font color=orange size=5>Your Code, Documentation, Ideas and Timer - All Start Here...</font></strong>

**Student's Section** (between ⏳ symbols): add your code and documentation here.

In [ ]:
tY

,city-mpg,highway-mpg,price
823666,36.84,47.19,5167.33
70066,18.63,42.68,21200.19
...,...,...,...
305711,27.59,37.27,9625.36
985772,24.05,26.03,9312.02


## **Task 1. Preprocessing Pipeline**

Explain elements of your preprocessing pipeline i.e. feature engineering, subsampling, clustering, dimensionality reduction, etc.
1. Why did you choose these elements? (Something in EDA, prior experience,...? Btw, EDA is not required)
1. How do you evaluate the effectiveness of these elements?
1. What else have you tried that worked or didn't?

**Student's answer:**

Firstly, we did explanatory research of data. In particular, we reviewed the size of dataset and analyzed the correlation among features, which could be the issue for linear models, and among the features and each of the target variables.

We measured all the performance shifts using R^2, however, most of the preprocessing approaches failed (Scalers, PCA, PolynomialFeatures, subsampling and permutations of these approaches) as they either lowered the score, did not make any difference, or were conflicting with limits (time limit and collab RAM limit)

EDA showed showed that some columns (for example compression-ratio peak-rpm) had very low correlation to desired targets, however we did not find appropriate use of freed resources by ommiting these columns (without score performance hit), so we did not touch dataframe structure.

We decided to use SplineTransformer, as it made the score higher and did not brake the competition limits. We experimented with parameters and found best fit (approx 58 seconds in collab)

## **Task 2. Modeling Approach**
Explain your modeling approach, i.e. ideas you tried and why you thought they would be helpful.

1. How did these decisions guide you in modeling?
1. How do you evaluate the effectiveness of these elements?
1. What else have you tried that worked or didn't?

**Student's answer:**

We tried multiple models, such as DecisionTreeRegressor, RidgeCV, LinearRegression, etc, but in the end we used the combination of model and parameters presented below. DecisionTreeRegressor gave magnificient results, however we realised that the results were too good to be true, and indeed, performance on the training set was excellent, however splitting the data to predict on not previously seen data gave horrible results.

Additionally, we realised that parameter n_jobs could help us, as nproc of collab system returned 2 (we think that since n_jobs can be beneficial, if the system has n>=2 threads, which is the case)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import SplineTransformer

vxindex = vX.index

s = SplineTransformer(n_knots=12, degree=2)
tX = s.fit_transform(tX)
vX = s.transform(vX)

In [ ]:
m = MultiOutputRegressor(LinearRegression(n_jobs=2)).fit(tX, tY)
print(f'In-sample R^2 = {m.score(tX,tY):.4f}')

In-sample R^2 = 0.3632


In-sample R^2 = 0.3595
Output R^2 = 0.3591
5, 3

In [ ]:
pY = pd.DataFrame(m.predict(vX), index=vxindex, columns=YCols)
ToCSV(pY, 'Auto_spline')

# **References:**

1. Remember to cite your sources here as well! At the least, your textbook should be cited. Google Scholar allows you to effortlessly copy/paste an APA citation format for books and publications. Also cite StackOverflow, package documentation, and other meaningful internet resources to help your peers learn from these (and to avoid plagiarism claims).

<font size=5>⌛</font> <strong><font color=orange size=5>Do not exceed competition's runtime limit!</font></strong>

<hr color=red>


In [ ]:
tmr.ShowTime()    # measure Colab's runtime. Do not remove. Keep as the last cell in your notebook.

Runtime is 58 sec


## 💡**Starter Ideas**

1. Tune model hyperparameters and try different allowed models
1. Try to linear and non-linear feature normalization: shift/scale, log, divide features by features (investigate scatterplot matrix)
1. Try higher order feature interactions and polynomial features on a small subsample. Then identify key features or select key principal components. The final model can be trained on a larger or even full training sample. You can use [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) to reduce the feature set
1. Do a thorough EDA: look for feature augmentations that result in linear decision boundaries between pairs of classes.
1. Evaluate predictions and focus on poorly predicted "groups":
  1. Strongest errors. E.g. the model is very confident about the wrong label
1. Do scatter plots show piecewise linear shape? Can a separate linear model be used on each support, or can the pattern be linearized via transformations?
1. Try modeling each output separately from inputs or from a other modeled output
1. Try stepwise selection and regularization and remove "unimportant" features from final model